In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd drive/My \Drive/NLP

In [ ]:
!pip install textattack==0.3.7

In [ ]:
!pip install lime

In [ ]:
pip install -U datasets

In [ ]:
# Load libraries
import pandas as pd
import numpy as np
import nltk
import torch
import torch.nn as nn
import datasets
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
from lime.lime_text import LimeTextExplainer
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import random
import transformers
import textattack
from textattack.datasets import Dataset
from datasets import load_dataset
from transformers import GPT2Tokenizer
from textattack.models.wrappers import ModelWrapper
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack import Attack
from textattack.transformations import WordSwapEmbedding
from textattack.search_methods import GreedyWordSwapWIR
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.constraints.semantics import WordEmbeddingDistance
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import datasets

nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

In [ ]:
torch.manual_seed(42)
np.random.seed(42)

# Load the dataset
dataset = datasets.load_dataset('ucberkeley-dlab/measuring-hate-speech', 'default')
df = dataset['train'].to_pandas()
df.rename(columns={'text': 'tweet', 'hate_speech_score': 'class'}, inplace=True)
df['binary_class'] = df['class'] > 0.5

# Convert to lowercase, remove punctuation, extra spaces, URLs, mentions, and hashtags
df['tweet'] = df['tweet'].str.lower().replace(r'[^\w\s]', '', regex=True).replace(' {2,}', ' ', regex=True).replace('"', '')
df['tweet'] = df['tweet'].replace(r'http\S+|www.\S+|@\w+|#\w+', '', regex=True)

# Tokenization
nltk.download('punkt')
df['tweet'] = df['tweet'].apply(nltk.word_tokenize)

# Lemmatization
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
df['tweet'] = df['tweet'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

# Removing stop-words
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
df['tweet'] = df['tweet'].apply(lambda x: ' '.join([word for word in x if word not in stop_words]))

# Prepare data for DataLoader
max_length = 50
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
df['tweet'] = df['tweet'].apply(lambda x: tokenizer.tokenize(x)[:max_length])

# Create word2index dictionary
word2index = {}
for tweet in df['tweet']:
    for word in tweet:
        if word not in word2index:
            word2index[word] = len(word2index)

# Convert words to indices and pad sequences
input_data = np.zeros((len(df['tweet']), max_length), dtype=int)
for i, tweet in enumerate(df['tweet']):
    for j, word in enumerate(tweet):
        input_data[i, j] = word2index[word]

# Encode labels
le = LabelEncoder()
y = le.fit_transform(df['binary_class'])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(input_data, y, test_size=0.3, stratify=y, random_state=42)

class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.tensor(self.X[idx], dtype=torch.long), torch.tensor(self.y[idx], dtype=torch.long)

train_dataset = TextDataset(X_train, y_train)
test_dataset = TextDataset(X_test, y_test)

batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class TextCNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes):
        super(TextCNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.conv1 = nn.Conv2d(1, 100, (3, embed_dim))
        self.conv2 = nn.Conv2d(1, 100, (4, embed_dim))
        self.conv3 = nn.Conv2d(1, 100, (5, embed_dim))
        self.fc = nn.Linear(300, num_classes)

    def forward(self, x):
        x = self.embedding(x).unsqueeze(1)
        x1 = F.relu(self.conv1(x)).squeeze(3)
        x2 = F.relu(self.conv2(x)).squeeze(3)
        x3 = F.relu(self.conv3(x)).squeeze(3)
        x1 = F.max_pool1d(x1, x1.size(2)).squeeze(2)
        x2 = F.max_pool1d(x2, x2.size(2)).squeeze(2)
        x3 = F.max_pool1d(x3, x3.size(2)).squeeze(2)
        x = torch.cat((x1, x2, x3), 1)
        x = self.fc(x)
        return x

vocab_size = len(word2index)
embed_dim = 100
num_classes = 2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TextCNN(vocab_size, embed_dim, num_classes).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_dataloader)}")

    # Save Model
    torch.save(model, './Weights/HuggingFaceCNN.pth')

model = torch.load('./Weights/HuggingFaceCNN.pth')

# Test the model
# Prepare true labels and predictions for metrics calculation
true_labels = []
predictions = []

with torch.no_grad():
    for inputs, labels in test_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        true_labels.extend(labels.cpu().numpy())
        predictions.extend(predicted.cpu().numpy())

# Calculate accuracy, precision, recall, F1-score, and confusion matrix
accuracy = accuracy_score(true_labels, predictions)
precision, recall, f1_score, _ = precision_recall_fscore_support(true_labels, predictions, average='weighted')
conf_mat = confusion_matrix(true_labels, predictions)

print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1-score: ", f1_score)
print("Confusion Matrix:\n", conf_mat)

In [ ]:
# Wrapper for the TextCNN model
class LSTMBaselineWrapper:
    def __init__(self, model):
        self.model = model

    def __call__(self, text_input_list):
        preds = []
        for text in text_input_list:
            input_tensor = tokenize_and_pad([text]).long()
            output = self.model(input_tensor.to(device))
            pred = torch.softmax(output, dim=1).squeeze().tolist()
            preds.append(pred)
        return np.array(preds)

def tokenize_and_pad(text_list):
    max_length = 50
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    tokens = [tokenizer.tokenize(text)[:max_length] for text in text_list]
    token_indices = np.zeros((len(tokens), max_length), dtype=int)
    for i, tweet in enumerate(tokens):
        for j, word in enumerate(tweet):
            if word in word2index:
                token_indices[i, j] = word2index[word]
    return torch.tensor(token_indices)

wrapped_model = LSTMBaselineWrapper(model)
class_names = ['hate_speech', 'offensive_language', 'neither']

# Explainability
def lime_analysis(text, wrapped_model, class_names):
    explainer = LimeTextExplainer(class_names=class_names)
    exp = explainer.explain_instance(text, wrapped_model, num_features=10, num_samples=2)
    return exp.as_list()

df['tweet'] = df['tweet'].apply(lambda x: ' '.join(x))
text_to_explain = random.choice(df['tweet'])
print("Text to explain:", text_to_explain)
lime_results = lime_analysis(text_to_explain, wrapped_model, class_names)
print("LIME analysis results:")
print(lime_results)

def calculate_doe(lime_results):
    feature_scores = [abs(score) for _, score in lime_results]
    std_dev = np.std(feature_scores)
    significant_features = len([score for score in feature_scores if score > std_dev])
    return significant_features / len(feature_scores)

doe = calculate_doe(lime_results)
print("Degree of Explainability (DoE):", doe)

In [ ]:
# Define number of samples for analysis
num_sam = 20

# Load dataset
def load_custom_dataset(path):
    df = pd.read_csv(path)
    df = df.dropna(subset=['tweet', 'class'])
    return df

# LIME Analysis
def lime_analysis(text, wrapped_model, class_names):
    explainer = LimeTextExplainer(class_names=class_names)
    exp = explainer.explain_instance(text, wrapped_model, num_features=10, num_samples=2)
    return exp.as_list()

# Calculate Degree of Explainability (DoE)
def calculate_doe(lime_results):
    feature_scores = [abs(score) for _, score in lime_results]
    std_dev = np.std(feature_scores)
    significant_features = len([score for score in feature_scores if score > std_dev])
    return significant_features / len(feature_scores)

# Calculate average DoE for Multiple samples
def calculate_average_doe(df, wrapped_model, class_names, samples=num_sam):
    doe_values = []
    sample_texts = random.sample(list(df['tweet']), samples)
    for text in sample_texts:
        lime_results = lime_analysis(text, wrapped_model, class_names)
        doe = calculate_doe(lime_results)
        doe_values.append(doe)
    average_doe = np.mean(doe_values)
    return average_doe

# Assuming wrapped_model and class_names are defined elsewhere
class_names = ['Hate speech', 'Offensive language', 'Neutral']

# Calculate and print average DoE
average_doe = calculate_average_doe(df, wrapped_model, class_names)
print(f"Average Degree of Explainability (DoE) for {num_sam} samples:", average_doe)

In [ ]:
import numpy as np
from lime.lime_text import LimeTextExplainer
from nltk.corpus import wordnet
import random
import torch
import torch.nn.functional as F

# Function to get synonyms for a word
def get_synonym(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            if lemma.name() != word:
                synonyms.add(lemma.name().replace('_', ' '))
    return random.choice(list(synonyms)) if synonyms else word

# Function to generate adversarial example using LIME
def generate_adversarial_example(text, predictor, explainer, num_features=2):
    exp = explainer.explain_instance(text, predictor, num_features=num_features)
    words = text.split()
    for feature, _ in exp.as_list()[:num_features]:
        if feature in words:
            idx = words.index(feature)
            words[idx] = get_synonym(words[idx])
    return ' '.join(words)

# Wrapper for model prediction
def model_predict(texts):
    model.eval()
    indexed_texts = [[word2index.get(word, len(word2index)-1) for word in text.split()] for text in texts]
    padded_texts = torch.nn.utils.rnn.pad_sequence([torch.tensor(text) for text in indexed_texts], batch_first=True, padding_value=0)
    padded_texts = padded_texts.to(device)
    with torch.no_grad():
        outputs = model(padded_texts.long())
    return F.softmax(outputs, dim=1).cpu().numpy()

# LIME-based adversarial attack
def lime_based_attack(dataset, samples=num_sam):
    correct_before_attack = 0
    correct_after_attack = 0
    total_samples = 0

    explainer = LimeTextExplainer(class_names=['hate speech', 'offensive language', 'neither'])

    for inputs, labels in dataset:
        for input_tensor, label in zip(inputs, labels):
            if total_samples >= samples:
                return total_samples, correct_before_attack, correct_after_attack

            # Convert tensor to string
            text = ' '.join([list(word2index.keys())[i] for i in input_tensor.tolist() if i < len(word2index)])

            # Original prediction
            original_pred = model_predict([text])[0].argmax()
            if original_pred == label:
                correct_before_attack += 1

            # Generate adversarial example
            adv_text = generate_adversarial_example(text, model_predict, explainer)
            adv_pred = model_predict([adv_text])[0].argmax()

            if adv_pred == label:
                correct_after_attack += 1

            total_samples += 1

    return total_samples, correct_before_attack, correct_after_attack

# Perform the attack
total_samples, correct_before_attack, correct_after_attack = lime_based_attack(train_dataloader)

# Calculate metrics
accuracy_before_attack = correct_before_attack / total_samples
accuracy_after_attack = correct_after_attack / total_samples
adv_rob = accuracy_after_attack / accuracy_before_attack if accuracy_before_attack > 0 else 0

attack_resilience = adv_rob / average_doe if average_doe > 0 else 0

# Print results
print("LIME-based adversarial attack results:")
print(f"Total samples: {total_samples}")
print(f"Correct before attack: {correct_before_attack}")
print(f"Correct after attack: {correct_after_attack}")
print(f"Accuracy before attack: {accuracy_before_attack}")
print(f"Accuracy after attack: {accuracy_after_attack}")
print("")
print("Results: ")
print("Adversarial Robustness (AdvRob):", adv_rob)
print("Attack Resilience (Ar):", attack_resilience)